In [98]:
import pandas as pd
import os
from glob import glob
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.metrics import recall_score,precision_score,accuracy_score

In [16]:
mp_annot = pd.read_csv('mp_annotations.csv',index_col=0)
print(f"{mp_annot.shape[0]} rows in mp_annot")
files = glob(f"{os.pardir}{os.sep}course_dataset{os.sep}ASL*{os.sep}annotations.csv")
true_annot = pd.concat([pd.read_csv(f).set_index('ID') for f in files],ignore_index=True)
true_annot.drop('person_idx',axis=1,inplace=True)
print(f"{true_annot.shape[0]} rows in true_annot")

/Users/sprietthibault/miniforge3/envs/tf/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


1008500 rows in mp_annot
1074906 rows in true_annot


In [24]:
df = pd.merge(mp_annot,true_annot,on=["frame","gesture","joint","video_idx"])
df.head()

,frame,gesture,joint,video_idx,x_x,y_x,x_y,y_y
0,0,ASL_letter_A,root,0,342.496634,523.881149,323.633333,520.750000
1,0,ASL_letter_A,thumb_1,0,264.379234,453.056450,266.033333,454.616667
2,0,ASL_letter_A,thumb_2,0,225.868979,451.839333,230.833333,445.016667
3,0,ASL_letter_A,thumb_3,0,202.671032,472.358170,199.900000,466.350000
4,0,ASL_letter_A,index_1,0,229.475327,493.944626,0.000000,0.000000


In [33]:
X_train,X_val,X_test = [],[],[]
y_train,y_val,y_test = [],[],[]

for label in df.gesture.unique():
    df = df.loc[df.gesture == label]
    video_idx = df.video_idx.unique()
    train,test = train_test_split(video_idx,test_size=.2)
    train,val = train_test_split(train,test_size=.2)
    for idx in video_idx:
        frames = df.loc[df.video_idx == idx]
        coordinates_X = frames[["x_x","y_x"]].to_numpy().reshape(len(frames.frame.unique()),40).tolist()
        coordinates_y = frames[["x_y","y_y"]].to_numpy().reshape(len(frames.frame.unique()),40).tolist()
        if idx in train:
            X_train+= coordinates_X
            y_train+= coordinates_y
        elif idx in val:
            X_val+= coordinates_X
            y_val+= coordinates_y

        else:
            assert idx in test
            X_test+= coordinates_X
            y_test+= coordinates_y

X_train,X_val,X_test = np.array(X_train),np.array(X_val),np.array(X_test)
y_train,y_val,y_test = np.array(y_train),np.array(y_val),np.array(y_test)

In [79]:
def euclidean_distance_loss(y_true, y_pred):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

model = Sequential(
    layers=[
        Input(shape=(40,)),
        Dense(1024),
        Dense(1024),
        Dense(1024),
        Dense(1024),
        Dense(40,activation='relu')
    ]
)
model.compile(optimizer=Adamax(learning_rate=1e-4),loss=euclidean_distance_loss)
earlyStop = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-3,
    patience=5
)

In [80]:
history = model.fit(
    x=X_train,y=y_train,batch_size=25,epochs=100,validation_data=(X_val,y_val)
)

Epoch 1/100
216/216 [==============================] - 2s 7ms/step - loss: 899.7087 - val_loss: 875.7277
Epoch 2/100
216/216 [==============================] - 1s 6ms/step - loss: 806.1906 - val_loss: 829.2652
Epoch 3/100
216/216 [==============================] - 1s 6ms/step - loss: 803.2036 - val_loss: 861.2900
Epoch 4/100
216/216 [==============================] - 1s 7ms/step - loss: 803.3894 - val_loss: 905.6494
Epoch 5/100
216/216 [==============================] - 1s 6ms/step - loss: 725.4840 - val_loss: 784.5983
Epoch 6/100
216/216 [==============================] - 1s 6ms/step - loss: 640.3568 - val_loss: 700.5961
Epoch 7/100
216/216 [==============================] - 1s 7ms/step - loss: 567.4111 - val_loss: 487.1414
Epoch 8/100
216/216 [==============================] - 1s 6ms/step - loss: 441.3905 - val_loss: 445.1917
Epoch 9/100
216/216 [==============================] - 1s 6ms/step - loss: 445.0602 - val_loss: 432.8632
Epoch 10/100
216/216 [==============================] -

In [110]:
model.save('models/transfered')

/Users/sprietthibault/miniforge3/envs/tf/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning:

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.

/Users/sprietthibault/miniforge3/envs/tf/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:1397: UserWarning:

`layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.




FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: models/transfered/assets


In [81]:
history_df = pd.DataFrame(data=history.history)
fig = px.line(history_df,y=["loss","val_loss"])
fig.show()

In [86]:
y_pred = model.predict(y_test)
y_test_flatten = y_test.reshape(-1)
y_pred_flatten = y_pred.reshape(-1)

In [109]:
TP = sum((y_pred_flatten == 0) & (y_test_flatten == 0))
TN = sum((y_pred_flatten != 0) & (y_test_flatten != 0))
FP = sum((y_pred_flatten == 0) & (y_test_flatten != 0))
FN = sum((y_pred_flatten != 0) & (y_test_flatten == 0))

p = TP  / (TP+FP)
r = TP / (TP + FN)
a = (TP + TN) / (TP+TN+FP+FN)

print("Model performance to predict coordinates 0")
print(f"\taccuracy : {a*100:.1f} %")
print(f"\tprecision : {p*100:.1f} %")
print(f"\trecall : {r*100:.1f} %")

Model performance to predict coordinates 0
	accuracy : 94.3 %
	precision : 94.0 %
	recall : 81.4 %
